In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Input, BatchNormalization
from glob import glob

Using TensorFlow backend.


In [2]:
train = '/home/hasan/DATA SET/cell-images-for-detecting-malaria/train/'
test = '/home/hasan/DATA SET/cell-images-for-detecting-malaria/test/'

In [3]:
folder = glob('/home/hasan/DATA SET/cell-images-for-detecting-malaria/train/*')

# Defining Model

In [4]:
base_model = VGG19(input_shape=(128,128,3), include_top=False, weights='imagenet')
x = base_model.output
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(300)(x)
x = Dense(350, activation='relu')(x)
predictions = Dense(len(folder), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# Summary of the Model

In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

# Compiling The Model

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Declaring ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(  
                      shear_range=0.2, 
                      zoom_range=0.2,  
                      horizontal_flip=True,                
                      rescale=1./255, 
                      )

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_data_generator = train_datagen.flow_from_directory('/home/hasan/DATA SET/cell-images-for-detecting-malaria/train/',
                                                  target_size=(128,128),
                                                  batch_size=32,
                                                  class_mode='categorical')

test_data_generator = test_datagen.flow_from_directory('/home/hasan/DATA SET/cell-images-for-detecting-malaria/test/',
                                                      target_size=(128,128),
                                                      batch_size=32,
                                                      class_mode='categorical'
                                                      )

Found 27558 images belonging to 2 classes.
Found 27558 images belonging to 2 classes.


# Fitting Train Data

In [ ]:
model_fit = model.fit_generator(generator=train_data_generator, 
              steps_per_epoch=len(train_data_generator),
              epochs=5,   
              validation_data=test_data_generator, 
              validation_steps=len(test_data_generator)
              )

Epoch 1/5
862/862 [==============================] - 4071s 5s/step - loss: 0.4044 - accuracy: 0.8678 - val_loss: 0.2357 - val_accuracy: 0.8656
Epoch 2/5
862/862 [==============================] - 4137s 5s/step - loss: 0.2276 - accuracy: 0.9118 - val_loss: 0.0118 - val_accuracy: 0.9341
Epoch 3/5
595/862 [===================>..........] - ETA: 11:06 - loss: 0.2009 - accuracy: 0.9228

# Accuracy and Loss graph of Training and Testing Data

In [ ]:
accuracy = model_fit.history['accuracy']
val_accuracy = model_fit.history['val_accuracy']

loss = model_fit.history['loss']
val_loss = model_fit.history['val_loss']

epochs = range(len(val_accuracy))
plt.plot(epochs, accuracy, 'go', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'go', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='validation loss')
plt.title('training and validation loss')
plt.legend()
plt.show()